In [1]:
import pandas as pd
import numpy as np
import os
import PIL
from pathlib import Path
from PIL import Image
# import scipy
import IPython
from IPython.display import display
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Convolution2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical
import tensorflow as tf
        

In [2]:

root_path = Path().resolve()
img = Image.open(f'{root_path}/data/training/aeroplane/2007_000033.jpg')
print(img.size)
# img
maxPixel = 128
imScalled = img.resize((maxPixel,maxPixel))
img_pixel = np.reshape(list(imScalled.getdata()),(maxPixel,maxPixel,3))
img_pixel
print(img_pixel.size)

(500, 366)
49152


In [38]:
maxPixel=128
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
valid_datagen = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(os.path.join(root_path, 'data/training'),  batch_size=200, class_mode = 'binary', target_size = (img.size[0], img.size[1]),classes=['aeroplane','others'])
valid_generator = valid_datagen.flow_from_directory(os.path.join(root_path, 'data/validation'),  batch_size=200, class_mode = 'binary', target_size = (img.size[0], img.size[1]),classes=['aeroplane','others'])

#train_generator = train_datagen.flow_from_directory(os.path.join(root_path, 'data/training'),  batch_size=20, class_mode = 'binary', target_size = (maxPixel, maxPixel),classes=['aeroplane','others'])
#valid_generator = valid_datagen.flow_from_directory(os.path.join(root_path, 'data/validation'),  batch_size=20, class_mode = 'binary', target_size = (maxPixel, maxPixel),classes=['aeroplane','others'])

base_model = InceptionV3(input_shape = (img.size[0], img.size[1], 3), include_top = False, weights = 'imagenet', classes=20)
#base_model = InceptionV3(input_shape = (maxPixel, maxPixel, 3), include_top = False, weights = 'imagenet', classes=20)

for layer in base_model.layers:
    layer.trainable = False


Found 83 images belonging to 2 classes.
Found 723 images belonging to 2 classes.


In [39]:
base_model.output

<KerasTensor: shape=(None, 14, 10, 2048) dtype=float32 (created by layer 'mixed10')>

In [40]:
x = Flatten()(base_model.output)
x = Dense(1000, activation='relu')(x)
#x = Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
#x = Dense(1000, activation='relu')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [42]:
print(Flatten()(model.output))
500*366*3

KerasTensor(type_spec=TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None), name='flatten_14/Reshape:0', description="created by layer 'flatten_14'")


549000

In [24]:
model.output

<KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'dense_3')>

In [43]:
# Fit the model
plane_detector = model.fit(
    train_generator, 
    validation_data=valid_generator, 
    steps_per_epoch=1,
#     validation_steps=valid_generator.samples/valid_generator.batch_size,
    epochs=10
)

Epoch 1/10
1/1 [==============================] - 114s 114s/step - loss: 5.3998 - acc: 0.0000e+00 - val_loss: 0.5780 - val_acc: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 99s 99s/step - loss: 5.5218 - acc: 0.0000e+00 - val_loss: 1.0450 - val_acc: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 100s 100s/step - loss: 5.4853 - acc: 0.0000e+00 - val_loss: 1.0217 - val_acc: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 101s 101s/step - loss: 5.4458 - acc: 0.0000e+00 - val_loss: 0.8831 - val_acc: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 102s 102s/step - loss: 5.1775 - acc: 0.0000e+00 - val_loss: 0.8480 - val_acc: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 101s 101s/step - loss: 5.0035 - acc: 0.0000e+00 - val_loss: 0.7787 - val_acc: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 98s 98s/step - loss: 4.8789 - acc: 0.0000e+00 - val_loss: 0.7144 - val_acc: 0.0000e+00
Epoch 8/10
1/1 [===============

In [49]:
for inputs_batch, labels_batch in train_generator:
    features_batch = model.predict(inputs_batch)
    print(inputs_batch)
    print(labels_batch)
    print(features_batch)
    break

3/3 [==============================] - 29s 9s/step
[[[[0.6353687  0.6705137  0.7058824 ]
   [0.63222694 0.66445386 0.7058824 ]
   [0.6359474  0.6627451  0.7081698 ]
   ...
   [0.92549026 0.8941177  0.80392164]
   [0.92549026 0.8941177  0.80392164]
   [0.9298778  0.8985053  0.8095628 ]]

  [[0.63772154 0.66816086 0.7058824 ]
   [0.6345798  0.66915953 0.7058824 ]
   [0.631438   0.662876   0.7058824 ]
   ...
   [0.9301849  0.8988123  0.80995756]
   [0.9570117  0.92563915 0.84444916]
   [0.97842294 0.9470504  0.87204826]]

  [[0.6392157  0.6666667  0.7058824 ]
   [0.6369327  0.6689498  0.7058824 ]
   [0.63379085 0.6675817  0.7058824 ]
   ...
   [0.97824746 0.9468749  0.871829  ]
   [0.9629179  0.9315454  0.852667  ]
   [0.9490197  0.9176471  0.8352942 ]]

  ...

  [[0.88856626 0.86880517 0.7772856 ]
   [0.88473386 0.8553918  0.7792018 ]
   [0.882353   0.8470589  0.7803922 ]
   ...
   [0.8299104  0.8070246  0.7446888 ]
   [0.87988394 0.8378527  0.768203  ]
   [0.9039223  0.8436979  0.780112

In [64]:
features = np.zeros(shape=(83, 1000))  # Must be equal to the output of the convolutional base
labels = np.zeros(shape=(83))

# Pass data through convolutional base
i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = model.predict(inputs_batch)
    print(features_batch.shape)
    features[i * 200: (i + 1) * 200] = features_batch
    labels[i * 200: (i + 1) * 200] = labels_batch
    i += 1
    if i * 200 >= 83:
        break

3/3 [==============================] - 31s 10s/step
(83, 1000)


In [67]:
train

array([[14.68993664,  0.        , 14.28793812, ...,  9.15919971,
         0.        , 12.22969055],
       [16.53250694,  0.        , 16.57906532, ..., 13.09087753,
         0.40081367, 15.69054222],
       [13.81405926,  4.33168507, 14.15189648, ..., 10.27601624,
         4.48490477, 12.74816799],
       ...,
       [21.35681152,  3.65161252, 24.48859024, ..., 16.89321899,
         6.20963001, 21.91637802],
       [29.12653732,  7.53386021, 28.50595474, ..., 20.99895287,
         7.35999012, 26.73404312],
       [12.38465118,  0.        , 11.89767361, ...,  7.50119686,
         0.        , 13.13149548]])

In [54]:
for inputs_batch, labels_batch in train_generator:
    print(inputs_batch.shape)
#     print(model.predict(inputs_batch).shape)

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 128, 3)
(83, 128, 

KeyboardInterrupt: 